# Setup

In [1]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt

# Question 1

In [2]:
img = cv.imread('winter.jfif', 0)

eq = cv.equalizeHist(img)
eq2 = cv.equalizeHist(eq)

cv.imshow('result', np.hstack((img, eq, eq2)))
cv.waitKey(0)
cv.destroyAllWindows()

Comment on question 1: No difference between both output

# Question 2

In [3]:
img = cv.imread('electronic.jfif', 0)

sobelx = cv.Sobel(img, cv.CV_64F, 1, 0, ksize = 1)
sobely = cv.Sobel(img, cv.CV_64F, 0, 1, ksize = 1)

grad_mag_L2 = cv.magnitude(sobelx, sobely)
grad_mag_L2 = cv.convertScaleAbs(grad_mag_L2)

cv.imshow('result', np.hstack((img, grad_mag_L2)))
cv.waitKey(0)
cv.destroyAllWindows()

Kernel 1 size is the most appropriate because output with less noise

In [4]:
img_blur = cv.GaussianBlur(img, (5, 5), 0)

sobelx_blur = cv.Sobel(img_blur, cv.CV_64F, 1, 0, ksize = 1)
sobely_blur = cv.Sobel(img_blur, cv.CV_64F, 0, 1, ksize = 1)

grad_mag_L2_blur = cv.magnitude(sobelx_blur, sobely_blur)
grad_mag_L2_blur = cv.convertScaleAbs(grad_mag_L2_blur)

cv.imshow('result', np.hstack((img, grad_mag_L2, grad_mag_L2_blur)))
cv.waitKey(0)
cv.destroyAllWindows()

The noises reduced after apply Gaussian blur

In [5]:
laplacian = cv.Laplacian(img_blur, cv.CV_64F, ksize = 3)

laplacian_8u = cv.convertScaleAbs(laplacian)

cv.imshow('result', laplacian_8u)
cv.waitKey(0)
cv.destroyAllWindows()

Optimal pathway is gaussian smoothing then sobel edge detection because the output have the minimal noises.

# Question 3

In [6]:
img = cv.imread('pineapple.jfif', 0)
img_blur = cv.GaussianBlur(img, (5,5), 0)

# Sobel
sobelx = cv.Sobel(img_blur, cv.CV_64F, 1,0, ksize = 3)
sobely = cv.Sobel(img_blur, cv.CV_64F, 0,1, ksize = 3)
blur_grad_mag_L2 = cv.magnitude(sobelx,sobely)
blur_grad_mag_L2 = cv.convertScaleAbs(blur_grad_mag_L2)

# Laplacian
laplacian_blur = cv.Laplacian(img_blur, cv.CV_64F, ksize = 3)
laplacian_blur = np.uint8(np.absolute(laplacian_blur))

# Prewitt
kernelx = np.array([[1,1,1],[0,0,0],[-1,-1,-1]])
kernely = np.array([[-1,0,1],[-1,0,1],[-1,0,1]])
img_prewittx = cv.filter2D(img_blur, -1, kernelx)
img_prewitty = cv.filter2D(img_blur, -1, kernely)
img_comb_prewitt = (img_prewittx + img_prewitty)

# Scharr Derivatives
scharr_X = cv.Scharr(img_blur, cv.CV_64F, 1, 0) 
scharr_X_abs = np.uint8(np.absolute(scharr_X)) 
scharr_Y = cv.Scharr(img_blur, cv.CV_64F, 0, 1) 
scharr_Y_abs = np.uint8(np.absolute(scharr_Y)) 
scharr_XY_combined = cv.bitwise_or(scharr_Y_abs,scharr_X_abs) 

# Canny operators
edges = cv.Canny(img_blur, 100, 200)

# Display
cv.imshow("result", np.hstack((blur_grad_mag_L2,laplacian_blur, img_comb_prewitt, scharr_XY_combined, edges)))
cv.waitKey(0)
cv.destroyAllWindows()

- Sobel: result with most edges detected
- Laplacian: edge that is simlar contrast hard to detect
- Prewitt:  result same as sobel but less edge detected on the same colour
- Scharr: result with all edges detected
- Canny: Cleanest result  

# Question 4

In [7]:
# Read and convert image to gray
img = cv.imread('electronic.jfif')
img_gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY) 

clahe = cv.createCLAHE(clipLimit = 8.0, tileGridSize = (15, 15))
dst = clahe.apply(img_gray)

# Smooth image
blur = cv.GaussianBlur(dst, (7, 7), 0)
ret, th = cv.threshold(blur, 180, 255, cv.THRESH_BINARY)

# Find laptop contour
contour, hierarchy = cv.findContours(th, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)

for i in contour:
    area = cv.contourArea(i)
    if area >= 1000:
        cnt = i
        x, y, w, h = cv.boundingRect(cnt)
        cv.rectangle(img, (x,y), (x+w, y+h), (0, 255, 0), 2)
    else:
        continue

cv.imshow('result', img)
cv.waitKey(0)
cv.destroyAllWindows()